In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
# dimension = 20
def load_data(train_filename,test_filename):
    X_train = []
    X_test = []
    Y_train = []
    Y_test = []
    
    with open(train_filename,'r') as train_file:
        for line in train_file:
            temp = line.strip(' ').split(' ')
            X_train.append(temp[:-1])
            Y_train.append(temp[-1])
    with open(test_filename,'r') as test_file:
        for line in test_file:
            temp = line.strip(' ').split(' ')
            X_test.append(temp[:-1])
            Y_test.append(temp[-1])

    temp_train = np.ones((len(X_train))).reshape((-1,1))
    temp_test = np.ones((len(X_test))).reshape((-1,1))
    
    return np.hstack([temp_train,np.array(X_train,dtype=np.float64)]),np.hstack([temp_test,np.array(X_test,dtype=np.float64)]),np.array(Y_train,dtype=np.float64),np.array(Y_test,dtype=np.float64)

In [3]:
X_train,X_test,Y_train,Y_test = load_data('HW4_13_train.txt','HW4_13_test.txt')

In [8]:
X_train.shape

(200, 3)

In [11]:
def train_arg(X, Y, lamda):
    return np.linalg.inv((X.T.dot(X)+lamda*np.identity(X.shape[1]))).dot(X.T).dot(Y)

In [37]:
def get_error_single(X_train, Y_train, X_test, Y_test, lamda):
    w = train_arg(X_train, Y_train, lamda)
    predict_train = np.sign(X_train.dot(w))
    predict_test = np.sign(X_test.dot(w))
    
    return np.mean(predict_train != Y_train), np.mean(predict_test != Y_test)

In [26]:
Ein, Eout = get_error_single(X_train, Y_train, X_test, Y_test, 10)

In [27]:
print(Ein)
print(Eout)

0.05
0.045


In [59]:
def get_error(X_train, Y_train, X_test, Y_test, lamda_list, choose_by = 'Ein'):
    opt_ein = 1
    opt_eout = 1
    opt_lambda = 0
    
    for l in lamda_list:
        ein, eout = get_error_single(X_train, Y_train, X_test, Y_test, l)
        if(choose_by == 'Ein'):
            if(ein < opt_ein):
                opt_ein = ein
                opt_eout = eout
                opt_lambda = l
        else:
            if(eout < opt_eout):
                opt_ein = ein
                opt_eout = eout
                opt_lambda = l
    return opt_lambda, opt_ein, opt_eout

In [30]:
lamda_list = [10**i for i in range(2,-11,-1)]
opt_lambda, opt_ein, opt_eout = get_error(X_train, Y_train, X_test, Y_test, lamda_list)
print(opt_lambda)
print(opt_ein)
print(opt_eout)

1e-08
0.015
0.02


In [32]:
lamda_list = [10**i for i in range(2,-11,-1)]
opt_lambda, opt_ein, opt_eout = get_error(X_train, Y_train, X_test, Y_test, lamda_list , 'Eout')
print(opt_lambda)
print(opt_ein)
print(opt_eout)

1e-07
0.03
0.015


In [36]:
def get_error_val_single(X_train_l, Y_train_l, X_val, Y_val, X_test, Y_test, lamda):
    w = train_arg(X_train_l, Y_train_l, lamda)
    predict_train_l = np.sign(X_train_l.dot(w))
    predict_val = np.sign(X_val.dot(w))
    predict_test = np.sign(X_test.dot(w))
    
    return np.mean(predict_train_l != Y_train_l), np.mean(predict_val != Y_val), np.mean(predict_test != Y_test)

In [68]:
def get_error_val(X_train, Y_train, X_test, Y_test, lamda_list, choose_by = 'Ein'):
    opt_ein = 1
    opt_eout = 1
    opt_lambda = 0
    opt_eval = 1
    
    X_train_l = X_train[:120]
    X_val = X_train[120:]
    
    Y_train_l = Y_train[:120]
    Y_val = Y_train[120:]
    
    for l in lamda_list:
        ein, e_val, eout = get_error_val_single(X_train_l, Y_train_l, X_val, Y_val, X_test, Y_test, l)
        if(choose_by == 'Ein'):
            if(ein < opt_ein):
                opt_ein = ein
                opt_eval = e_val
                opt_eout = eout
                opt_lambda = l
        elif(choose_by == 'Eval'):
            if(e_val < opt_eval):
                opt_ein = ein
                opt_eval = e_val
                opt_eout = eout
                opt_lambda = l
        else:
            if(eout < opt_eout):
                opt_ein = ein
                opt_eval = e_val
                opt_eout = eout
                opt_lambda = l
    return opt_lambda, opt_ein, opt_eval, opt_eout

In [69]:
lamda_list = [10**i for i in range(2,-11,-1)]
opt_lambda, opt_ein, opt_eval, opt_eout = get_error_val(X_train, Y_train, X_test, Y_test, lamda_list)
print(opt_lambda)
print(opt_ein)
print(opt_eval)
print(opt_eout)

1e-08
0.0
0.05
0.025


In [70]:
lamda_list = [10**i for i in range(2,-11,-1)]
opt_lambda, opt_ein, opt_eval, opt_eout = get_error_val(X_train, Y_train, X_test, Y_test, lamda_list, 'Eval')
print(opt_lambda)
print(opt_ein)
print(opt_eval)
print(opt_eout)

1
0.03333333333333333
0.0375
0.028


In [72]:
ein, eout = get_error_single(X_train,Y_train,X_test,Y_test,1)
print(ein)
print(eout)

0.035
0.02


In [110]:
def get_error_5cv_single(X_train, Y_train,X_test, Y_test, lamda):
    e_val = 0
    for i in range(5):
        X_train_l = np.vstack([X_train[:i*40],X_train[(i+1)*40:]])
        X_val = X_train[i*40:(i+1)*40]

        Y_train_l = np.append(Y_train[:i*40],Y_train[(i+1)*40:])
        Y_val = Y_train[i*40:(i+1)*40]
        
        w = train_arg(X_train_l, Y_train_l, lamda)
        predict_val = np.sign(X_val.dot(w))
        
        e_val += np.mean(predict_val != Y_val)
        
    return e_val/5

In [111]:
def get_error_5cv(X_train, Y_train, X_test, Y_test, lamda_list):
    opt_lambda = 0
    opt_eval = 1

    for l in lamda_list:
        e_val = get_error_5cv_single(X_train, Y_train,X_test, Y_test, l)
        if(e_val < opt_eval):
                opt_eval = e_val
                opt_lambda = l
    return opt_lambda, opt_eval

In [112]:
lamda_list = [10**i for i in range(2,-11,-1)]
opt_lambda, opt_eval = get_error_5cv(X_train, Y_train, X_test, Y_test, lamda_list)
print(opt_lambda)
print(opt_eval)

1e-08
0.03


In [113]:
ein, eout = get_error_single(X_train,Y_train,X_test,Y_test,1e-08)
print(ein)
print(eout)

0.015
0.02
